In [7]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_probability as tfp

tfd = tfp.distributions  # pylint: disable=invalid-name
tfb = tfp.bijectors  # pylint: disable=invalid-name

In [18]:
tf.experimental.numpy.experimental_enable_numpy_behavior()

In [3]:
from iflow.integration import integrator
from iflow.integration import couplings

In [16]:
def variance_weighted_result(means, stddevs):
    """ Computes weighted mean and stddev of given means and
        stddevs arrays, using Inverse-variance weighting
    """
    assert np.size(means) == np.size(stddevs)
    assert means.shape == stddevs.shape
    variance = 1./np.sum(1./stddevs**2, axis=-1)
    mean = np.sum(means/(stddevs**2), axis=-1)
    mean *= variance
    return mean, np.sqrt(variance)

def train_iflow(integrate, ptspepoch, epochs):
    """ Run the iflow integrator

    Args:
        integrate (Integrator): iflow Integrator class object
        ptspepoch (int): number of points per epoch in training
        epochs (int): number of epochs for training

    Returns:
        numpy.ndarray(float): value of loss (mean) and its uncertainty (standard deviation)

    """
    means = np.zeros(epochs)
    stddevs = np.zeros(epochs)
    for epoch in range(epochs):
        loss, integral, error = integrate.train_one_step(ptspepoch,
                                                         integral=True)
        means[epoch] = integral
        stddevs[epoch] = error
        _, current_precision = variance_weighted_result(means[:epoch+1], stddevs[:epoch+1])
        if epoch % 10 == 0:
            print('Epoch: {:3d} Loss = {:8e} Integral = '
                  '{:8e} +/- {:8e} Total uncertainty = {:8e}'.format(epoch, loss,
                                                                     integral, error,
                                                                     current_precision))

In [19]:
def func(args):
    return tf.exp(-1*tf.sqrt(args.T[0]**2 + args.T[1]**2))
    #return args.T[0]**3 + torch.exp(-10* args.T[1]**2)
    
n_dims = 2

low = -2*np.ones(n_dims, dtype=np.float64)
high = 2*np.ones(n_dims, dtype=np.float64)
dist = tfd.Uniform(low=low, high=high)
dist = tfd.Independent(distribution=dist,
                        reinterpreted_batch_ndims=1)

In [20]:
optimizer = tf.keras.optimizers.Adam(3e-4, clipnorm=10.0)
integrate = integrator.Integrator(func, dist, optimizer, loss_func='exponential')

In [21]:
train_iflow(integrate, 10000, 10)

ValueError: in user code:

    File "c:\DevSw\miniforge3\envs\normflows\Lib\site-packages\iflow\integration\integrator.py", line 96, in train_one_step  *
        self.optimizer.apply_gradients(
    File "c:\DevSw\miniforge3\envs\normflows\Lib\site-packages\keras\src\optimizers\base_optimizer.py", line 268, in apply_gradients  **
        grads, trainable_variables = zip(*grads_and_vars)

    ValueError: not enough values to unpack (expected 2, got 0)
